## Health utils

In [5]:
from pprint import pprint

# Health utils

def print_member_status(client):
    db = client.admin
    rs_status = db.command({'replSetGetStatus': 1})

    for m in rs_status['members']:
        print(m['name'], m['stateStr'])

def print_client_info(client):
    db = client.admin
    server_info = db.command('serverStatus')
    # pprint(server_info)

    print('MongoDB version:', server_info['version'])
    print('MongoDB process:', server_info['process'])
    print('MongoDB uptime:', server_info['uptime'], 'seconds')
    print('MongoDB uptime:', server_info['uptimeMillis'], 'milliseconds')
    print('MongoDB connections:', server_info['connections']['current'])  # current connections
    print('MongoDB max connections:', server_info['connections']['available'])  # max connections
    print('MongoDB storage engine:', server_info['storageEngine']['name'])

# Database setup

### following pymongo setup

https://pymongo.readthedocs.io/en/stable/examples/high_availability.html

In [2]:
### run multiple mongod processes

!mkdir -p data/db0 data/db1 data/db2

In [ ]:
# $ mongod --port 27017 --dbpath data/db0 --replSet foo

# $ mongod --port 27018 --dbpath data/db1 --replSet foo

# $ mongod --port 27019 --dbpath data/db2 --replSet foo 


In [3]:
from pymongo import MongoClient

c = MongoClient('localhost', 27017, directConnection=True)

In [6]:
print_client_info(c)

MongoDB version: 8.0.9
MongoDB process: mongod
MongoDB uptime: 53.0 seconds
MongoDB uptime: 49001 milliseconds
MongoDB connections: 9
MongoDB max connections: 838851
MongoDB storage engine: wiredTiger


In [7]:
!mongosh --eval "rs.status()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000MongoServerError: no replset config has been received


In [6]:
print_member_status(c)

OperationFailure: no replset config has been received, full error: {'ok': 0.0, 'errmsg': 'no replset config has been received', 'code': 94, 'codeName': 'NotYetInitialized'}

In [12]:
# print info about client to check if its a single node or replica set or sharded cluster

def print_client_type(client):

    db = client.admin
    server_info = db.command('serverStatus')
    # pprint(server_info)

    if 'setName' in server_info:
        print('MongoDB is a replica set')
    elif 'shardName' in server_info:
        print('MongoDB is a sharded cluster')
    else:
        print('MongoDB is a standalone instance')

print_client_type(c)

MongoDB is a standalone instance


In [14]:
db = c.admin
server_info = db.command('serverStatus')
server_info['service']

['shard']

In [16]:
server_info['service']

['shard']

In [8]:
# This is similar to mongosh rs.initiate()

# Initialize the replica set with a configuration

config = {'_id': 'foo', 'members': [
    {'_id': 0, 'host': 'localhost:27017'},
    {'_id': 1, 'host': 'localhost:27018'},
    {'_id': 2, 'host': 'localhost:27019'}]}

c.admin.command("replSetInitiate", config)

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1746794710, 1),
  'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'keyId': 0}},
 'operationTime': Timestamp(1746794710, 1)}

In [18]:
client = MongoClient('mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=foo')

In [19]:
client

MongoClient(host=['localhost:27019', 'localhost:27017', 'localhost:27018'], document_class=dict, tz_aware=False, connect=True, replicaset='foo')

In [20]:
print_client_type(client)

MongoDB is a standalone instance


In [11]:
print_member_status(c)

localhost:27017 PRIMARY
localhost:27018 SECONDARY
localhost:27019 SECONDARY


In [22]:
db = client.admin
server_info = db.command('serverStatus')
server_info['service']

['shard']

In [25]:
server_info

{'host': 'DESKTOP-QJASGSB',
 'version': '8.0.9',
 'process': 'mongod',
 'service': ['shard'],
 'pid': 109191,
 'uptime': 389.0,
 'uptimeMillis': 386117,
 'uptimeEstimate': 386,
 'localTime': datetime.datetime(2025, 5, 9, 6, 45, 12, 795000),
 'asserts': {'regular': 0,
  'warning': 0,
  'msg': 0,
  'user': 1468,
  'tripwire': 0,
  'rollovers': 0},
 'batchedDeletes': {'batches': 0,
  'docs': 0,
  'stagedSizeBytes': 0,
  'timeInBatchMillis': 0,
  'refetchesDueToYield': 0},
 'catalogStats': {'collections': 0,
  'capped': 0,
  'clustered': 0,
  'timeseries': 0,
  'views': 0,
  'internalCollections': 18,
  'internalViews': 1,
  'csfle': 0,
  'queryableEncryption': 0},
 'changeStreamPreImages': {'purgingJob': {'totalPass': 39,
   'docsDeleted': 0,
   'bytesDeleted': 0,
   'scannedCollections': 39,
   'scannedInternalCollections': 0,
   'maxStartWallTimeMillis': 0,
   'timeElapsedMillis': 0},
  'numDocs': 0,
  'totalBytes': 0,
  'docsInserted': 0,
  'storageSize': 4096,
  'freeStorageSize': 0},

In [24]:
!mongosh --eval "rs.status()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000{
  set: 'foo',
  date: ISODate('2025-05-09T06:46:05.385Z'),
  myState: 1,
  term: Long('1'),
  syncSourceHost: '',
  syncSourceId: -1,
  heartbeatIntervalMillis: Long('2000'),
  majorityVoteCount: 2,
  writeMajorityCount: 2,
  votingMembersCount: 3,
  writableVotingMembersCount: 3,
  optimes: {
    lastCommittedOpTime: { ts: Timestamp({ t: 1746773160, i: 1 }), t: Long('1') },
    lastCommittedWallTime: ISODate('2025-05-09T06:46:00.081Z'),
    readConcernMajorityOpTime: { ts: Timestamp({ t: 1746773160, i: 1 }), t: Long('1') },
    appliedOpTime: { ts: Timestamp({ t: 1746773160, i: 1 }), t: Long('1') },
    durableOpTime: { ts: Timestamp({ t: 1746773160, i: 1 }), t: Long('1') },
    writtenOpTime: { ts: Timestamp({ t: 1746773160, i: 1 }), t: Long('1') },
    lastAppliedWallTime: ISODate('2025-05-09T06:46:00.081Z'),
    lastDurableWallTime: ISODate('2025-05-09T06:46:00.081Z'),
    lastWrittenWallTi

In [23]:
server_info

{'host': 'DESKTOP-QJASGSB',
 'version': '8.0.9',
 'process': 'mongod',
 'service': ['shard'],
 'pid': 109191,
 'uptime': 389.0,
 'uptimeMillis': 386117,
 'uptimeEstimate': 386,
 'localTime': datetime.datetime(2025, 5, 9, 6, 45, 12, 795000),
 'asserts': {'regular': 0,
  'warning': 0,
  'msg': 0,
  'user': 1468,
  'tripwire': 0,
  'rollovers': 0},
 'batchedDeletes': {'batches': 0,
  'docs': 0,
  'stagedSizeBytes': 0,
  'timeInBatchMillis': 0,
  'refetchesDueToYield': 0},
 'catalogStats': {'collections': 0,
  'capped': 0,
  'clustered': 0,
  'timeseries': 0,
  'views': 0,
  'internalCollections': 18,
  'internalViews': 1,
  'csfle': 0,
  'queryableEncryption': 0},
 'changeStreamPreImages': {'purgingJob': {'totalPass': 39,
   'docsDeleted': 0,
   'bytesDeleted': 0,
   'scannedCollections': 39,
   'scannedInternalCollections': 0,
   'maxStartWallTimeMillis': 0,
   'timeElapsedMillis': 0},
  'numDocs': 0,
  'totalBytes': 0,
  'docsInserted': 0,
  'storageSize': 4096,
  'freeStorageSize': 0},

In [26]:
client.nodes

frozenset({('localhost', 27017), ('localhost', 27018), ('localhost', 27019)})

In [55]:
client = MongoClient('localhost', directConnection=True, replicaSet='foo')
print(client)

print_member_status(client)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, directconnection=True, replicaset='foo')
localhost:27017 PRIMARY
localhost:27018 (not reachable/healthy)
localhost:27019 SECONDARY


In [45]:
client = MongoClient('mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=foo')

In [53]:
print_member_status(client)

localhost:27017 PRIMARY
localhost:27018 SECONDARY
localhost:27019 SECONDARY


In [56]:

db = client.test
item = db.test.insert_one({"x": 1})

In [49]:
client = MongoClient("localhost", replicaSet='foo')
db = client.test
# item = db.test.insert_one({"x": 1})

In [51]:
item

InsertOneResult(ObjectId('681ca95d4eed2dbe0f9209d3'), acknowledged=True)

In [58]:
db.test.list_indexes()

In [68]:
db.client.address

('localhost', 27017)

In [67]:
db.test.find_one()

{'_id': ObjectId('681ca95d4eed2dbe0f9209d3'), 'x': 1}

In [ ]:
# kill the 'db.client.address' process which is primary node using pymongo
# client.admin.command('shutdown')


AutoReconnect: localhost:27019: connection closed (configured timeouts: connectTimeoutMS: 20000.0ms)

### following mongodb setup

https://www.mongodb.com/resources/products/compatibilities/deploying-a-mongodb-cluster-with-docker


1) Create docker network with 'docker network create mongoCluster'
2) Create mongod instances in docker with "docker run -d --rm -p 27017:27017 --name mongo1 --network mongoCluster mongo:5 mongod --replSet myReplicaSet --bind_ip localhost,mongo1"
3) Create 2 more such instances
4) Use pymongo to connect init the replica set

In [ ]:
# "# for mongo1
# docker run -d --rm -p 27017:27017 --name mongo1 --network mongoCluster mongo:5 mongod --replSet myReplicaSet --bind_ip localhost,mongo1
# for mongo2
# docker run -d --rm -p 27018:27017 --name mongo2 --network mongoCluster mongo:5 mongod --replSet myReplicaSet --bind_ip localhost,mongo2
# for mongo3
# docker run -d --rm -p 27019:27017 --name mongo3 --network mongoCluster mongo:5 mongod --replSet myReplicaSet --bind_ip localhost,mongo3

In [4]:
from pymongo import MongoClient

client = MongoClient('mongo1', 27017, directConnection=True)
client

MongoClient(host=['mongo1:27017'], document_class=dict, tz_aware=False, connect=True, directconnection=True)

In [5]:
client.nodes

frozenset()

In [6]:
!docker exec -it mongo1 mongosh --eval "rs.status()"


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000MongoServerError: no replset config has been received


In [ ]:
# This is similar to mongosh rs.initiate()

# Initialize the replica set with a configuration

from pymongo import MongoClient

client = MongoClient('localhost', 27017, directConnection=True)

config = {'_id': 'myReplicaSet', 'members': [
    {'_id': 0, 'host': 'mongo1:27017'},
    {'_id': 1, 'host': 'mongo2:27017'},
    {'_id': 2, 'host': 'mongo3:27017'}
]}
 
client.admin.command("replSetInitiate", config)

NameError: name 'c' is not defined

In [ ]:
print_member_status(client)

KeyboardInterrupt: 

In [153]:
!docker exec -it mongo1 mongosh ping mongo2

]0;mongosh mongodb://127.0.0.1:27017/ping?directConnection=true&serverSelectionTimeoutMS=2000Error: ENOENT: no such file or directory, open '/mongo2'


In [154]:
!docker exec -it mongo1 mongosh --eval "rs.status()"

]0;mongosh mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000{
  set: 'myReplicaSet',
  date: ISODate('2025-05-09T08:45:39.088Z'),
  myState: 2,
  term: Long('0'),
  syncSourceHost: '',
  syncSourceId: -1,
  heartbeatIntervalMillis: Long('2000'),
  majorityVoteCount: 2,
  writeMajorityCount: 2,
  votingMembersCount: 3,
  writableVotingMembersCount: 3,
  optimes: {
    lastCommittedOpTime: { ts: Timestamp({ t: 1746780333, i: 1 }), t: Long('-1') },
    lastCommittedWallTime: ISODate('2025-05-09T08:45:33.505Z'),
    readConcernMajorityOpTime: { ts: Timestamp({ t: 1746780333, i: 1 }), t: Long('-1') },
    appliedOpTime: { ts: Timestamp({ t: 1746780333, i: 1 }), t: Long('-1') },
    durableOpTime: { ts: Timestamp({ t: 1746780333, i: 1 }), t: Long('-1') },
    writtenOpTime: { ts: Timestamp({ t: 1746780333, i: 1 }), t: Long('-1') },
    lastAppliedWallTime: ISODate('2025-05-09T08:45:33.505Z'),
    lastDurableWallTime: ISODate('2025-05-09T08:45:33.505Z'),
    las

In [155]:
print_member_status(client)

mongo1:27017 PRIMARY
mongo2:27017 SECONDARY
mongo3:27017 SECONDARY


### Connect to the MongoDB Replica Set

In [156]:
def print_member_status(client):
    db = client.admin
    rs_status = db.command({'replSetGetStatus': 1})

    for m in rs_status['members']:
        print(m['name'], m['stateStr'])

client = MongoClient('localhost', 27017, directConnection=True, replicaSet='myReplicaSet')

print_member_status(client)


mongo1:27017 PRIMARY
mongo2:27017 SECONDARY
mongo3:27017 SECONDARY


In [157]:

client = MongoClient('localhost:27018', replicaSet='myReplicaSet', directConnection=True)

print_member_status(client)


mongo1:27017 PRIMARY
mongo2:27017 SECONDARY
mongo3:27017 SECONDARY


In [158]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=myReplicaSet')

print_member_status(client)
status = client.admin.command('replSetGetStatus')

for member in status['members']:
    print(member['name'])


ServerSelectionTimeoutError: mongo1:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo2:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 681dc0cf7185250e101043e6, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>, <ServerDescription ('mongo2', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongo2:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: soc

In [159]:
client = MongoClient('localhost:27017', directConnection=True)
cfg = client.admin.command('replSetGetConfig')
pprint(cfg['config']['members'])


[{'_id': 0,
  'arbiterOnly': False,
  'buildIndexes': True,
  'hidden': False,
  'host': 'mongo1:27017',
  'priority': 1.0,
  'secondaryDelaySecs': 0,
  'tags': {},
  'votes': 1},
 {'_id': 1,
  'arbiterOnly': False,
  'buildIndexes': True,
  'hidden': False,
  'host': 'mongo2:27017',
  'priority': 1.0,
  'secondaryDelaySecs': 0,
  'tags': {},
  'votes': 1},
 {'_id': 2,
  'arbiterOnly': False,
  'buildIndexes': True,
  'hidden': False,
  'host': 'mongo3:27017',
  'priority': 1.0,
  'secondaryDelaySecs': 0,
  'tags': {},
  'votes': 1}]


In [112]:
client = MongoClient('mongodb://localhost:27017,localhost:27018,localhost:27019/?replicaSet=myReplicaSet')

client

MongoClient(host=['localhost:27019', 'localhost:27017', 'localhost:27018'], document_class=dict, tz_aware=False, connect=True, replicaset='myReplicaSet')

In [124]:
client.admin.command('ping')

ServerSelectionTimeoutError: client is configured to connect to a replica set named 'myReplicaSet' but this node belongs to a set named 'None', Timeout: 30s, Topology Description: <TopologyDescription id: 681dbba47185250e101043d2, topology_type: Single, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=ConfigurationError("client is configured to connect to a replica set named 'myReplicaSet' but this node belongs to a set named 'None'")>]>

In [117]:

print("Databases:", client.list_database_names())

Databases: ['admin', 'config', 'local']


In [160]:
client.admin.command('ping')
# Example: List databases
print("Databases:", client.list_database_names())

# Select a database and collection (it will be created if it doesn't exist)
db = client["mydatabase"]
collection = db["mycollection"]

# Insert a document
doc_id = collection.insert_one({"name": "Test Document", "value": 123}).inserted_id
print(f"Inserted document with id: {doc_id}")

# Find the document
found_doc = collection.find_one({"_id": doc_id})
print(f"Found document: {found_doc}")

# Print how many documents are in the collection
count = collection.count_documents({})
print(f"Number of documents in collection: {count}")

Databases: ['admin', 'config', 'local']
Inserted document with id: 681dc1307185250e101043e8
Found document: {'_id': ObjectId('681dc1307185250e101043e8'), 'name': 'Test Document', 'value': 123}
Number of documents in collection: 1


In [164]:

count = collection.count_documents({})
print(f"Number of documents in collection: {count}")

Number of documents in collection: 1


In [162]:
print_member_status(client)

mongo1:27017 PRIMARY
mongo2:27017 SECONDARY
mongo3:27017 SECONDARY


In [ ]:
from pymongo import MongoClient

client = MongoClient('localhost', 27018, directConnection=True, replicaSet='myReplicaSet')
print(client)

print_member_status(client)

MongoClient(host=['localhost:27018'], document_class=dict, tz_aware=False, connect=True, directconnection=True, replicaset='myReplicaSet')
mongo1:27017 PRIMARY
mongo2:27017 SECONDARY
mongo3:27017 SECONDARY
